<h1>F20BC Coursework</h1>
<p style="font-size:15px;">This notebook implements and analyses the Biologically-Inspired Computation coursework, focusing on Artificial Neural Networks (ANNs) and Particle Swarm Optimization (PSO). It includes a multi-layer ANN architecture trained using PSO to solve a regression task predicting concrete compressive strength. Additionally, it explores the impact of hyperparameters on model performance through systematic experiments.</p>

**<p style="font-size:18px;">Data Preparation</p>**
This contains all the data preprocessing and cleaning</p>

In [218]:
import os
print(os.getcwd())

/Users/imanteh/f20bc_new/F20BC/Notebooks


In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [220]:
df = pd.read_csv("../Dataset/concrete_data.csv", skiprows=1)

In [221]:
df.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [222]:
# Viewing the dimensionality of the dataset
df.shape

(1030, 9)

In [223]:
df.dropna(inplace = True)
df.isnull().sum()

cement                           0
blast_furnace_slag               0
fly_ash                          0
water                            0
superplasticizer                 0
coarse_aggregate                 0
fine_aggregate                   0
age                              0
concrete_compressive_strength    0
dtype: int64

In [224]:
print(f'Duplicate rows: {df.duplicated().sum()}') #print the number of duplicate rows

Duplicate rows: 25


In [225]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

**<p style="font-size:18px;">Scaling and Encoding</p>**

In [226]:
print(df.head())
print(df.columns)

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  concrete_compressive_strength  
0            1040.0            676.0   28                          79.99  
1            1055.0            676.0   28                          61.89  
2             932.0            594.0  270                          40.27  
3             932.0            594.0  365                          41.05  
4             978.4            825.5  360                          44.30  
Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate ', 'age',
       'concret

In [227]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1] # All columns except the last
y = df.iloc[:, -1] # The last column

In [228]:
print(X.shape)
print(y.shape)

(1005, 8)
(1005,)


In [229]:
print(X.head())

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  
0            1040.0            676.0   28  
1            1055.0            676.0   28  
2             932.0            594.0  270  
3             932.0            594.0  365  
4             978.4            825.5  360  


**<p style="font-size:18px;">Split data into train set and perform Scaling</p>**

In [230]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [231]:
print("First 5 rows of scaled training data:")
print(X_train_scaled[:5])

First 5 rows of scaled training data:
[[-0.7036142   0.7362629  -0.8810702   0.1799759  -1.0226942   1.32341753
  -0.17765612 -0.62455318]
 [-0.83112259 -0.84582676  1.04788182 -0.72981642  0.63596651  1.35948653
   0.32102131  0.88513966]
 [ 0.43140366  1.06620495 -0.8810702   0.38479825 -0.18498677 -1.33280675
   0.00745899 -0.28365479]
 [-1.05233032  0.66864226  1.10026551 -0.30588178  0.28412939  0.42298068
  -0.33758549 -0.51092038]
 [-1.17404287 -0.84582676  1.31904441  0.5419875   0.50193332 -1.24005789
   1.18741037 -0.28365479]]


**<p style="font-size:20px;">Artificial Neural Network (ANN)</p>**
Below are the implementations of the ANN to train and predict concrete compressive strength

In [232]:
class activationFunction:
    def logisticFunction(x): #For calculating the sigmoid function that always returns a value between 0 to 1
        f=1 / (1 + np.exp(-x))
        return f

    def reluFunction(x):
        f=np.maximum(0, x)  #Returns a value between 0 to infinite
        return f

    def hyperbolicFunction(x):
        f=np.tanh(x)          
        return f

    def leakyReLU(x, alpha=0.01):
        f= np.maximum(x, alpha * x)   
        return f

    def linearFunction(x):
        return x     

# Creates a neural network based on the layerSize and activationFunction entered
class ArtificialNeuralNetwork:
    def __init__(self, layerSize, activationFunction):
        self.layerSize=layerSize
        self.activationFunction = activationFunction
        self.weights=[np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i]) for i in range(len(layer_sizes) - 1)]
        self.biases=[np.random.randn(1, layer_sizes[i + 1]) for i in range(len(layer_sizes) - 1)]

#Prints the layers and values of the neural network
    def visualizeLayers(self):
        for i in range(len(self.layerSize)-1):
            print("LAYER COUNT:  ", i,"    NODES: ",self.layerSize[i])
            print("  WEIGHT SHAPE:   ", self.weights[i].shape)
            print("  BIAS SHAPE:    ",self.biases[i].shape )
            print("  WEIGHTS :      ",self.weights[i])
            print("  BIASES        ",self.biases[i])

    def forwardPropagation(self, x):
        output=x
        for i in range(len(self.weights)):
          #Calculate the dot product of each value
            matrix_total = np.dot(output, self.weights[i]) + self.biases[i]
            output= self.activationFunction[i](matrix_total)
        return output

In [233]:
layer_sizes = [8, 16, 8, 1]

activation_functions = [
    activationFunction.logisticFunction,
    activationFunction.reluFunction,
    activationFunction.linearFunction,
]
ann = ArtificialNeuralNetwork(layer_sizes, activation_functions)
ann.visualizeLayers()

x_example = np.random.rand(8, 8)  #8 samples, 8 features
output = ann.forwardPropagation(x_example)
print("ANN Output (Sample Predictions):", output)

LAYER COUNT:   0     NODES:  8
  WEIGHT SHAPE:    (8, 16)
  BIAS SHAPE:     (1, 16)
  WEIGHTS :       [[ 0.53544147  0.15846533 -0.31025214  0.35287494  0.58691551  0.1664136
  -0.37513465 -0.1015947  -0.3307024   0.27243421 -0.2570581   0.66821638
  -0.09467319  0.16291008 -0.60226634  0.55574995]
 [ 0.08798548  0.55141609  0.32116212 -0.38738469  0.47895543  0.61667488
  -0.14243891  0.74615848  0.83944895  0.4362373  -0.306065    0.12790472
  -0.01511931  0.99922949 -0.34703912 -1.72416207]
 [-0.47501467  0.78667344  0.0543111  -0.25141063  0.04958206  0.25714198
   0.46695049 -0.07091666 -0.44355414  0.39503609 -1.15305369  1.0908308
   0.07214873 -0.5336249   0.56865176  0.28718899]
 [-0.29162077 -0.1319626  -1.15153799  0.02710345 -0.44464136  0.35720007
  -0.77422751  0.00905314  0.10087358  0.03176892  0.24371878 -0.25761751
   0.27872676 -1.01949947 -0.39259603 -0.79001421]
 [ 0.12054502 -0.27787547  0.5226084  -0.19969323  0.05500788 -0.54461156
   0.50816027  0.73414498  0.0

**<p style="font-size:18px;">Loss Function</p>**
Since the problem domian is regression, MAE is utilised as the loss function

In [234]:
# Declaring the loss function - mean absolute error
class lossFunction:
    def evaluate(self,y_pred,y_train):
        self.y_pred=y_pred
        self.y_train=y_train

class MeanAbsoluteError(lossFunction):
    def evaluate(self, y_pred, y_train):
        f= np.mean(np.abs(y_pred - y_train))
        return f

In [235]:
# Convert from pandas to numpy array
y_train = y_train.to_numpy().reshape(-1, 1)
y_pred = ann.forwardPropagation(X_train_scaled)

# Verifying the shapes
print("y_pred shape:", y_pred.shape)
print("y_train shape:", y_train.shape)

# Used to calculate the loss function
loss_function=MeanAbsoluteError()
loss=loss_function.evaluate(y_pred, y_train)
print("Loss=", loss)

y_pred shape: (703, 1)
y_train shape: (703, 1)
Loss= 34.0209418232084


**<p style="font-size:20px;">Particle Swarm Optimisation (PSO)</p>**
Particle Swarm Opitmisation (PSO) is implemented here. 

In [236]:
# Initialising the particle object
class Particle:
    def __init__(self,vectorSize):
            
            self.particlePosition=np.random.rand(vectorSize) #initial position of the particle
            self.particleVelocity=np.random.rand(vectorSize) #initial velocity of the particle
        
            self.bestPosition=np.copy(self.particlePosition)
            self.informants=[]   # array to store all the informants of the particle

This particleToANn function converts a particle's position vector into the weights and biases of a neural network based on activation functions, and returns the neural network object.

In [237]:
# Code to convert the particle to an ANN
def particleToAnn(particle, annLayers, activationFunctions):
    
    neuralNetwork = ArtificialNeuralNetwork(layerSize=annLayers, activationFunction=activationFunctions)
    weightBiasIndexCount = 0
    
    for i in range(len(annLayers) - 1):
        # input for each neuron layer
        prevValue = annLayers[i]
        
        # output for each neuron layer
        nextValue = annLayers[i + 1]
        
        # mutliplying the layer counts
        weightRange = prevValue * nextValue
        
        # calculate weights
        weight = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + weightRange].reshape((prevValue, nextValue))
        
        weightBiasIndexCount += weightRange
        biases = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + nextValue].reshape((1, nextValue))
        weightBiasIndexCount += nextValue
        
        # setting the activationFunctions for the particle's ANN
        activation = activationFunctions[i]
        
        # setting the weights and biases for the particle's ANN
        neuralNetwork.weights[i] = weight
        neuralNetwork.biases[i] = biases
    return neuralNetwork

'''  referenced from: https://pyswarms.readthedocs.io/en/latest/examples/usecases/train_neural_network.html'''

'  referenced from: https://pyswarms.readthedocs.io/en/latest/examples/usecases/train_neural_network.html'

This assessFitness function evaluates the fitness of a particle by converting it into a neural network and also performs forward propagation. It also returns the loss

In [238]:
def assessFitness(particle, dataset, annLayers, activationFunctions, loss_function):
    x, y = dataset
    ann = particleToAnn(particle, annLayers, activationFunctions)  # converting the ann to a particle
    predictions = ann.forwardPropagation(x)  # perform forward propagation 
    loss = loss_function.evaluate(predictions, y.reshape(-1, 1))  # calculate the loss

    return loss

This PSO class implements the PSO algorithm by managing a swarm of particles, their informants, and also updates the velocity to find the optimal particle positions based on the fitness value, and global best position.

In [239]:
class ParticleSwarmOptimisation:
    # initialise the class variables
    def __init__(self, swarmSize, alpha, beta, delta, gamma, jumpSize, informantCount, vectorSize):
        self.swarmSize = swarmSize
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.gamma =gamma
        self.jumpSize = jumpSize
        self.informantCount = informantCount
        self.vectorSize = vectorSize
        self.global_best = None
        self.global_best_fitness = float('inf')

    # initialise the informants for each particle
    def initInformants(self, informantCount, particleArray):
        for p in particleArray:
            # blank array for informants
            informants=[]
            # iterate every particle in the array again
            for p in particleArray:
                # to store every particle except itself
                potentialInformants=[]
                # to check if the array is the array itself
                for potInf in particleArray:
                    if potInf!=p:
                        # assign to potential particle informant
                        potentialInformants.append(potInf)
                for i in range(informantCount):
                     informants.append(random.choice(potentialInformants))
                p.informants=informants  # assign all informants to the the particle

    def get_best_informant(self, particle, dataset, annLayers, activationFunctions, loss_function):
        bestInf = None
        bestFitnessInf = float('-inf')
        for i in particle.informants:
            # Assess the fitness of each informant
            fitness = assessFitness(i, dataset, annLayers, activationFunctions, loss_function)
            
            if fitness >  bestFitnessInf:
                bestFitnessInf = fitness
                bestInf = i
        return bestInf.particlePosition
        
    # Optimisation method
    def psoOptimisation(self, swarmSize, alpha, beta, gamma, jumpSize, informantCount, vectorSize,
                        dataset, annLayers, activationFunctions, loss_function, max_iterations=100):

        particleArray=[]
        # creating particles
        for i in range(swarmSize):
                particleArray.append(Particle(vectorSize))

        self.initInformants(informantCount, particleArray)
        best = None
        iteration = 0
        while iteration < max_iterations:
            # Update best particle
            for p in particleArray:
                # assessing fitness for p
                particleFitness = assessFitness(p, dataset, annLayers, activationFunctions, loss_function)
                # assigning the particle as the best
                if best is None or particleFitness < assessFitness(best, dataset, annLayers, activationFunctions, loss_function):
                    best = p
            # calculate the velocity
            for p in particleArray:
                # initialising the values
                previousBest = p.bestPosition
                informantsBest = self.get_best_informant(p, dataset, annLayers, activationFunctions, loss_function)
                allBest = best.bestPosition
                b = np.random.uniform(0.0, beta)
                c = np.random.uniform(0.0, gamma)
                d = np.random.uniform(0.0, delta)
                
                # calculate the new velocity
                newVelocity = (
                    alpha * p.particleVelocity +
                    b * (previousBest - p.particlePosition) +
                    c * (informantsBest - p.particlePosition) +
                    d * (allBest - p.particlePosition)
                )
                print("updated v:",  newVelocity)
                p.particleVelocity =  newVelocity
                p.particlePosition += jumpSize * newVelocity
                
            iteration += 1
        return best.particlePosition

<p style="font-size:18px;">Test out PSO Functionality</p>

In [240]:
def flatten(annLayers):
    sumValue=0
    for i in range(len(annLayers) - 1):
        weights = annLayers[i] * annLayers[i + 1]
        biases = annLayers[i+1]
        sumValue+=weights+biases
    return sumValue

# Code to test out PSO functionality
X_train_scaled = np.random.rand(10, 8) #training features
y_train = np.random.rand(10, 1)  # training labels

# defining the  hyperparameters for PSO
swarmSize = 10
alpha = 0.9  # inertia weight
beta = 0.7   # cognitive parameter
gamma = 0.5  # social parameter
delta = 0.5
jumpSize = 0.5
informantCount = 2
annLayers=layer_sizes
vectorSize = flatten(annLayers)
max_iterations = 10

# initialize the PSO optimiser class
pso = ParticleSwarmOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    delta = delta,
    gamma = gamma,
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize
)

# add it into the psoOptimisation and run
best_parameters = pso.psoOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    gamma = delta,  # Using delta as gamma as per your code
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize,

    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activation_functions,
    loss_function= MeanAbsoluteError(),
    max_iterations = max_iterations
)

print("Optimised ANN Parameters:", best_parameters)

updated v: [ 0.25442342  0.74307568  0.31254376  0.60221658  0.55259594  0.45659593
  0.06605162  0.87083187  0.18631997  0.81206582  0.57536364  0.05469385
  0.88957646  0.43855045  0.81217309  0.47458532  0.25920112  0.25245441
  0.3908553   0.14934652  0.37513326  0.44940222  0.27704916  0.58087358
  0.74951026  0.75579029  0.32864394  0.26671494  0.17655545  0.19381624
  0.63919446  0.48513737  0.2708533   0.67739581  0.462443   -0.00392329
  0.00843819  0.88966785  0.72797466  0.82980702  0.79902872  0.49506097
  0.73014142  0.30909308  0.32391565  0.89566864  0.02429747  0.03613755
  0.81038733  0.23381574  0.34553781  0.27404326  0.76716745  0.38041714
  0.61623617  0.44212788  0.76331866  0.04984245  0.80166736  0.42448436
  0.27292799  0.41692727  0.71691465  0.05279133  0.79254474  0.75771784
  0.64531694  0.12578892  0.77929991  0.21031489  0.53117464  0.1855347
  0.33844238  0.64586349  0.38459788  0.37109135  0.31217564  0.01588781
  0.40841034  0.40219219  0.17961646  0.6

<p style="font-size:18px;">Get Fitness Value</p>
The final fitness represents the performance of the ANN after its weights and biases have been optimised by the PSO

In [241]:
optimisingParticle = Particle(vectorSize)
optimisingParticle.particlePosition = best_parameters # assigning to the particle

# Constructing a new ann
newAnn = particleToAnn(
    optimisingParticle,
    annLayers,
    activation_functions
)
# Run through ANN with the optimised parameters
fitness_value = assessFitness(
    optimisingParticle, # evaluate fitness for the particle
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activation_functions,
    loss_function = MeanAbsoluteError()
)
print("Fitness Value:", fitness_value)

Fitness Value: 6.883061387356998


<p style="font-size:18px;">Training the ANN</p>

In [242]:
def evaluatingANN(best_parameters, X_test_scaled, y_test, annLayers, activation_functions):
    # Create a dummy particle with the optimized parameters
    optimized_particle = Particle(vectorSize=len(best_parameters))
    optimized_particle.particlePosition = best_parameters
    
    newAnn = particleToAnn(
    optimisingParticle,
    annLayers,
    activation_functions
    )
    # forward propagate the testing data through the ANN
    predictions=newAnn.forwardPropagation(X_test_scaled)
    
    # Converting y_test into a numpy array and reshape it
    y_test_array = y_test.to_numpy().reshape(-1, 1)
    loss_function = MeanAbsoluteError()
    test_loss = loss_function.evaluate(predictions, y_test_array)
    print(f"Testing MAE with optimized parameters: {test_loss}")

# testing code
evaluatingANN(
    best_parameters=best_parameters,
    X_test_scaled=X_test_scaled,
    y_test=y_test,
    annLayers=layer_sizes,
    activation_functions=activation_functions
)

Testing MAE with optimized parameters: 30.257046252521025
